Mickael CHENG, Kenny TANCHALEUNE

In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
#from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import nltk

In [2]:
df = pd.read_csv("text_emotion.csv")

In [3]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [4]:
import nltk
from nltk.tokenize import WordPunctTokenizer

tokenizer = nltk.RegexpTokenizer(r"\w+")



df['content'] = df['content'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [5]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')
df['content'] = df['content'].apply(lambda x: [item for item in x if item not in stopWords])

In [6]:
df['words'] = 1

In [7]:
#à tester
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

for index,entry in enumerate(df['content']):
    words = []
    
    for word in entry:
        pos_word = get_wordnet_pos(word)
        words.append(lemmatizer.lemmatize(word,pos_word))
    df.loc[index,'words'] = str(words)
    

In [8]:
df.head()

,tweet_id,sentiment,author,content,words
0,1956967341,empty,xoshayzers,"[tiffanylue, know, listenin, bad, habit, earli...","['tiffanylue', 'know', 'listenin', 'bad', 'hab..."
1,1956967666,sadness,wannamama,"[layin, n, bed, headache, ughhhh, waitin, call]","['layin', 'n', 'bed', 'headache', 'ughhhh', 'w..."
2,1956967696,sadness,coolfunky,"[funeral, ceremony, gloomy, friday]","['funeral', 'ceremony', 'gloomy', 'friday']"
3,1956967789,enthusiasm,czareaquino,"[wants, hang, friends, soon]","['want', 'hang', 'friend', 'soon']"
4,1956968416,neutral,xkilljoyx,"[dannycastillo, want, trade, someone, houston,...","['dannycastillo', 'want', 'trade', 'someone', ..."


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [10]:
df['sentiments'] = list(map(analyser.polarity_scores, df['words']))

In [11]:
df.head()

,tweet_id,sentiment,author,content,words,sentiments
0,1956967341,empty,xoshayzers,"[tiffanylue, know, listenin, bad, habit, earli...","['tiffanylue', 'know', 'listenin', 'bad', 'hab...","{'neg': 0.304, 'neu': 0.696, 'pos': 0.0, 'comp..."
1,1956967666,sadness,wannamama,"[layin, n, bed, headache, ughhhh, waitin, call]","['layin', 'n', 'bed', 'headache', 'ughhhh', 'w...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,1956967696,sadness,coolfunky,"[funeral, ceremony, gloomy, friday]","['funeral', 'ceremony', 'gloomy', 'friday']","{'neg': 0.672, 'neu': 0.328, 'pos': 0.0, 'comp..."
3,1956967789,enthusiasm,czareaquino,"[wants, hang, friends, soon]","['want', 'hang', 'friend', 'soon']","{'neg': 0.0, 'neu': 0.308, 'pos': 0.692, 'comp..."
4,1956968416,neutral,xkilljoyx,"[dannycastillo, want, trade, someone, houston,...","['dannycastillo', 'want', 'trade', 'someone', ...","{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp..."


In [12]:
def split_sentiment(col):
    NEG=[]
    POS=[]
    IS_POS=[]
    COMP=[]
    for entry in col:
        neg, neu, pos, comp = entry.values()
        NEG.append(neg)
        POS.append(pos)
        COMP.append(comp)
        
        
    return NEG, POS,COMP

df['neg'], df['pos'], df['comp'] = split_sentiment(df['sentiments'])

In [13]:
for element in df['comp']:
    if element>=0.05:
        df['comp'].replace(element,"Positive", inplace = True)
    if element<=-0.05:
        df['comp'].replace(element,"Negative", inplace = True)
    else:
        df['comp'].replace(element,"Neutral", inplace = True)

In [14]:
df.head()

,tweet_id,sentiment,author,content,words,sentiments,neg,pos,comp
0,1956967341,empty,xoshayzers,"[tiffanylue, know, listenin, bad, habit, earli...","['tiffanylue', 'know', 'listenin', 'bad', 'hab...","{'neg': 0.304, 'neu': 0.696, 'pos': 0.0, 'comp...",0.304,0.000,Negative
1,1956967666,sadness,wannamama,"[layin, n, bed, headache, ughhhh, waitin, call]","['layin', 'n', 'bed', 'headache', 'ughhhh', 'w...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,Neutral
2,1956967696,sadness,coolfunky,"[funeral, ceremony, gloomy, friday]","['funeral', 'ceremony', 'gloomy', 'friday']","{'neg': 0.672, 'neu': 0.328, 'pos': 0.0, 'comp...",0.672,0.000,Negative
3,1956967789,enthusiasm,czareaquino,"[wants, hang, friends, soon]","['want', 'hang', 'friend', 'soon']","{'neg': 0.0, 'neu': 0.308, 'pos': 0.692, 'comp...",0.000,0.692,Positive
4,1956968416,neutral,xkilljoyx,"[dannycastillo, want, trade, someone, houston,...","['dannycastillo', 'want', 'trade', 'someone', ...","{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...",0.000,0.178,Positive


In [15]:
df_new = pd.read_csv("text_emotion.csv")

In [16]:
df.rename(columns = {df.columns[5]: 'text'}, inplace = True)
df.rename(columns = {df.columns[0]: 'sentiments'}, inplace = True)
df.rename(columns = {df.columns[8]: 'label'}, inplace = True)
df['text'] = df_new['content']

In [17]:
df.head()

,sentiments,sentiment,author,content,words,text,neg,pos,label
0,1956967341,empty,xoshayzers,"[tiffanylue, know, listenin, bad, habit, earli...","['tiffanylue', 'know', 'listenin', 'bad', 'hab...",@tiffanylue i know i was listenin to bad habi...,0.304,0.000,Negative
1,1956967666,sadness,wannamama,"[layin, n, bed, headache, ughhhh, waitin, call]","['layin', 'n', 'bed', 'headache', 'ughhhh', 'w...",Layin n bed with a headache ughhhh...waitin o...,0.000,0.000,Neutral
2,1956967696,sadness,coolfunky,"[funeral, ceremony, gloomy, friday]","['funeral', 'ceremony', 'gloomy', 'friday']",Funeral ceremony...gloomy friday...,0.672,0.000,Negative
3,1956967789,enthusiasm,czareaquino,"[wants, hang, friends, soon]","['want', 'hang', 'friend', 'soon']",wants to hang out with friends SOON!,0.000,0.692,Positive
4,1956968416,neutral,xkilljoyx,"[dannycastillo, want, trade, someone, houston,...","['dannycastillo', 'want', 'trade', 'someone', ...",@dannycastillo We want to trade with someone w...,0.000,0.178,Positive


In [18]:
df.to_csv(r'Tweets_part.csv', index = False)